In [1]:
from __future__ import division
import os,time,cv2,scipy.io
import tensorflow as tf
# import tensorflow.contrib.slim as slim
import tf_slim as slim

import numpy as np
import matplotlib.pyplot as plt
# from discriminator import build_discriminator
import scipy.stats as st

In [2]:
# ENVIRONMENT
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import pathlib
from glob import glob

import warnings
warnings.filterwarnings("ignore")

# TENSORFLOW 2.0
import tensorflow as tf
print(tf.__version__)
print('GPU available:', tf.test.is_gpu_available())

from tensorflow.keras.models import Model
from tensorflow.python.keras.layers import Add, BatchNormalization, Conv2D, Dense, Flatten, Input, LeakyReLU, PReLU, Lambda, MaxPool2D
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanAbsoluteError, MeanSquaredError
binary_cross_entropy = BinaryCrossentropy()

from tensorflow.keras.metrics import Mean

import tensorflow.keras.backend as K
from tensorflow.python.data.experimental import AUTOTUNE

# ESSENTIAL 
import numpy as np
from sklearn.model_selection import train_test_split

# VISUALIZER
import matplotlib.pyplot as plt
plt.ioff()
from IPython import display

# UTILS
import time
from datetime import date
from tqdm import tqdm

2.1.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available: False


In [3]:
# L1 loss
def compute_l1_loss(in_img, out_img):
    return tf.reduce_mean(tf.abs(in_img - out_img))

# L1 loss on reflection image
## loss_l1_r = tf.where(issyn, compute_l1_loss(reflection_layer, reflection), 0)
# loss_l1_r = compute_l1_loss(reflection_layer, reflection)

In [4]:
# # Perceptual loss (get layer of VGG)
# def compute_percep_loss(input, output, reuse=False):
#     vgg_real=build_vgg19(output*255.0, reuse=reuse)
#     vgg_fake=build_vgg19(input*255.0, reuse=True)
#     p0=compute_l1_loss(vgg_real['input'],vgg_fake['input'])
#     p1=compute_l1_loss(vgg_real['conv1_2'],vgg_fake['conv1_2'])/2.6
#     p2=compute_l1_loss(vgg_real['conv2_2'],vgg_fake['conv2_2'])/4.8
#     p3=compute_l1_loss(vgg_real['conv3_2'],vgg_fake['conv3_2'])/3.7
#     p4=compute_l1_loss(vgg_real['conv4_2'],vgg_fake['conv4_2'])/5.6
#     p5=compute_l1_loss(vgg_real['conv5_2'],vgg_fake['conv5_2'])*10/1.5
#     return p0+p1+p2+p3+p4+p5


# # Perceptual Loss
# loss_percep_t = compute_percep_loss(transmission_layer, target)
# loss_percep_r = tf.where(issyn, compute_percep_loss(reflection_layer, reflection, reuse=True), 0.)
# loss_percep = tf.where(issyn, loss_percep_t+loss_percep_r, loss_percep_t)

In [68]:
# Pre-trained VGG 
from tensorflow.python.keras.applications.vgg19 import VGG19
vgg_19 = VGG19(input_shape=(None, None, 3), weights='imagenet', include_top=False)

In [64]:
# Perceptual loss (get layer of VGG)
def _vgg(output_layer):
    return Model(vgg_19.input, vgg_19.layers[output_layer].output)

block_vgg_0 = _vgg(0)
block_vgg_1 = _vgg(2)
block_vgg_2 = _vgg(5)
block_vgg_3 = _vgg(8)
block_vgg_4 = _vgg(13)
block_vgg_5 = _vgg(18)

block_vgg_list = []
for i in range(0, 7):
    block_vgg_list.append( eval('block_vgg_{}'.format(i)) )

In [66]:
block_vgg_list[2].summary()

Model: "model_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
Total params: 260,160
Trainable params: 260,160
Non-trainable params: 0
____________________________________________________

In [67]:
vgg_mean = [103.939, 116.779, 123.68]

def preprocess_vgg(in_img):
    in_img = tf.cast(in_img * 255., dtype=tf.float32)
    r, g, b = tf.split(in_img, 3, 3)
    bgr = tf.concat([b - vgg_mean[0],
                     g - vgg_mean[1],
                     r - vgg_mean[2]], axis=3)
    return bgr

def vgg_loss_block(in_img, block_num):
    img = preprocess_vgg(in_img)
    return block_vgg_list[block_num](img)

In [70]:
# def compute_percep_loss(in_img, out_img, reuse=False):
def compute_percep_loss(in_img, out_img):
    p0 = compute_l1_loss(vgg_loss_block(out_img, 0), vgg_loss_block(out_img, 0))             # Input, layer 0
    p1 = compute_l1_loss(vgg_loss_block(out_img, 2), vgg_loss_block(out_img, 2)) /2.6        # conv1_2, layer 2   
    p2 = compute_l1_loss(vgg_loss_block(out_img, 5), vgg_loss_block(out_img, 5)) /4.8        # conv2_2, layer 5 
    p3 = compute_l1_loss(vgg_loss_block(out_img, 8), vgg_loss_block(out_img, 8)) /3.7        # conv3_2, layer 8 
    p4 = compute_l1_loss(vgg_loss_block(out_img, 13), vgg_loss_block(out_img, 13)) /5.6      # conv4_2, layer 13 
    p5 = compute_l1_loss(vgg_loss_block(out_img, 18), vgg_loss_block(out_img, 18)) *10/1.5   # conv5_2, layer 18 

    return p0+p1+p2+p3+p4+p5

In [35]:
# Perceptual Loss
loss_percep_t = compute_percep_loss(transmission_layer, target)
# loss_percep_r = tf.where(issyn, compute_percep_loss(reflection_layer, reflection, reuse=True), 0.)
loss_percep_r = compute_percep_loss(reflection_layer, reflection)
# loss_percep = tf.where(issyn, loss_percep_t+loss_percep_r, loss_percep_t)
loss_percep = loss_percep_t + loss_percep_r

In [48]:
# vgg_18 = vgg_52(output_t)

# vgg_18

<tf.Tensor: shape=(1, 32, 32, 512), dtype=float32, numpy=
array([[[[  0.       ,   0.       ,  87.70332  , ...,   0.       ,
            0.       ,   0.       ],
         [  0.       ,   0.       ,   0.       , ...,   0.       ,
            0.       ,  31.945353 ],
         [  0.       ,  94.94548  ,  55.095768 , ...,   0.       ,
            0.       , 181.53154  ],
         ...,
         [  0.       , 270.8574   ,   0.       , ...,   0.       ,
            0.       ,   0.       ],
         [ 30.501225 , 194.59703  ,   0.       , ...,   0.       ,
            0.       ,   0.       ],
         [ 12.462572 ,  53.652515 ,  47.653477 , ...,   0.       ,
            0.       ,   0.       ]],

        [[ 34.578876 , 255.89658  ,  71.654274 , ...,   0.       ,
            0.       ,   0.       ],
         [  0.       , 173.34218  ,   0.       , ...,   0.       ,
            0.       ,   0.       ],
         [  0.       , 123.29019  ,  49.671616 , ...,   0.       ,
            0.       ,  71.

In [ ]:
EPS = 1e-12

# Adversarial Loss
predict_real = build_discriminator(input_img, target)
    
predict_fake = build_discriminator(input_img, transmission_layer)
    
# Compute GAN loss
d_loss = (tf.reduce_mean(-(tf.log(predict_real + EPS) + tf.log(1 - predict_fake + EPS)))) * 0.5
g_loss = tf.reduce_mean(-tf.log(predict_fake + EPS))

In [25]:
# Exclusion loss, in gradient domain
def compute_gradient(img):
    gradx = img[:,1:,:,:] - img[:,:-1,:,:]
    grady = img[:,:,1:,:] - img[:,:,:-1,:]
    return gradx, grady

def compute_exclusion_loss(img1, img2, level=1):
    gradx_loss=[]
    grady_loss=[]
    
    for l in range(level):
        gradx1, grady1 = compute_gradient(img1)
        gradx2, grady2 = compute_gradient(img2)
        alphax = 2.0* tf.reduce_mean(tf.abs(gradx1)) / tf.reduce_mean(tf.abs(gradx2))
        alphay = 2.0* tf.reduce_mean(tf.abs(grady1)) / tf.reduce_mean(tf.abs(grady2))
        
        gradx1_s = (tf.keras.activations.sigmoid(gradx1) *2) -1
        grady1_s = (tf.keras.activations.sigmoid(grady1) *2) -1
        gradx2_s=(tf.keras.activations.sigmoid(gradx2 *alphax) *2) -1
        grady2_s=(tf.keras.activations.sigmoid(grady2 *alphay) *2) -1

        gradx_loss.append( tf.reduce_mean( tf.multiply( tf.square(gradx1_s), tf.square(gradx2_s)) ) **0.25)
        grady_loss.append( tf.reduce_mean( tf.multiply( tf.square(grady1_s), tf.square(grady2_s)) ) **0.25)

#         img1= tf.nn.avg_pool(img1, [1,2,2,1], [1,2,2,1], padding='SAME')
#         img2= tf.nn.avg_pool(img2, [1,2,2,1], [1,2,2,1], padding='SAME')
        img1= tf.keras.layers.AveragePooling2D( pool_size= (2,2), padding='SAME')(img1)
        img2= tf.keras.layers.AveragePooling2D( pool_size= (2,2), padding='SAME')(img2)
        
    return gradx_loss, grady_loss

In [ ]:
# Gradient loss
# loss_gradx, loss_grady = compute_exclusion_loss(transmission_layer, reflection_layer, level=3)
# loss_gradxy = tf.reduce_sum(sum(loss_gradx) /3.) + tf.reduce_sum(sum(loss_grady) /3.)
## loss_grad = tf.where(issyn, loss_gradxy/2.0, 0)
# loss_grad = loss_gradxy / 2.0


# loss = loss_l1_r + loss_percep *0.2 + loss_grad

In [26]:
trans_test = cv2.imread('./Test-Images/back_2.jpg', -1)
blend_test = cv2.imread('./Test-Images/blended_2.jpg', -1)

##########
neww = 512
newh = 512

channel = 64
##########
output_t=cv2.resize(np.float32(trans_test),(neww,newh),cv2.INTER_CUBIC)/255.0
output_b=cv2.resize(np.float32(blend_test),(neww,newh),cv2.INTER_CUBIC)/255.0

#########
output_t = np.expand_dims(output_t,axis=0)
output_b = np.expand_dims(output_b,axis=0)

In [32]:
# Gradient loss
loss_gradx, loss_grady = compute_exclusion_loss(output_t, output_b, level=3)
loss_gradxy = tf.reduce_sum(sum(loss_gradx) /3.) + tf.reduce_sum(sum(loss_grady) /3.)

# issyn=tf.compat.v1.placeholder(tf.bool,shape=[])
# loss_grad = tf.where(issyn, loss_gradxy/2.0, 0)
loss_grad = loss_gradxy/2.0

In [33]:
loss_gradx, loss_grady
loss_gradxy, loss_grad

(<tf.Tensor: shape=(), dtype=float32, numpy=0.17865163>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.089325815>)